# 実験結果比較ノートブック

このノートブックでは、baseline / CLS Token / Multiscale の3つの実験結果を比較します。

## 使い方
1. 各実験を実行して結果を `exp_baseline`, `exp_cls`, `exp_multiscale` に保存
2. このノートブックで比較分析を実行

## Step 1: 結果ディレクトリの確認

In [ ]:
import os
import json
from pathlib import Path

# 実験ディレクトリの設定
experiments = {
    'baseline': '/content/exp_baseline',
    'cls_token': '/content/exp_cls',
    'multiscale': '/content/exp_multiscale'
}

# ディレクトリ構造を確認
for name, path in experiments.items():
    print(f"\n{'='*50}")
    print(f"📁 {name}: {path}")
    print('='*50)
    
    if os.path.exists(path):
        for root, dirs, files in os.walk(path):
            level = root.replace(path, '').count(os.sep)
            if level > 2:
                continue
            indent = '  ' * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = '  ' * (level + 1)
            for f in files[:5]:
                print(f"{subindent}{f}")
            if len(files) > 5:
                print(f"{subindent}... (and {len(files)-5} more)")
    else:
        print(f"  ❌ ディレクトリが存在しません")

## Step 2: メトリクスファイルの読み込み

In [ ]:
def find_metrics_files(directory):
    """メトリクスファイルを再帰的に検索"""
    if not os.path.exists(directory):
        return []
    
    metrics_files = []
    for root, dirs, files in os.walk(directory):
        for f in files:
            if f.startswith('metrics') and f.endswith('.json'):
                metrics_files.append(os.path.join(root, f))
    return metrics_files

def load_metrics(path):
    """メトリクスファイルを読み込み"""
    with open(path, 'r') as f:
        return json.load(f)

# 各実験のメトリクスを読み込み
results = {}

for name, directory in experiments.items():
    files = find_metrics_files(directory)
    print(f"\n{name}: {len(files)} 個のメトリクスファイル")
    
    if files:
        for f in files:
            print(f"  - {f}")
        
        # 最初のファイルを読み込み
        results[name] = load_metrics(files[0])
    else:
        print(f"  ❌ ファイルなし")

## Step 3: サマリー比較

In [ ]:
import pandas as pd

# サマリーメトリクスを抽出
summary_data = []

for name, metrics in results.items():
    row = {
        'Experiment': name,
        'AUROC (%)': metrics.get('mean_classification_au_roc', 0) * 100,
        'AP (%)': metrics.get('mean_classification_ap', 0) * 100,
        'F1 (%)': metrics.get('mean_classification_f1', 0) * 100,
    }
    
    # セグメンテーション（あれば）
    if 'mean_segmentation_au_roc' in metrics:
        row['Seg AUROC (%)'] = metrics['mean_segmentation_au_roc'] * 100
        row['AU-PRO (%)'] = metrics.get('mean_au_pro', 0) * 100
    
    summary_data.append(row)

if summary_data:
    df_summary = pd.DataFrame(summary_data)
    df_summary = df_summary.set_index('Experiment')
    
    print("\n" + "="*60)
    print("📊 実験サマリー")
    print("="*60)
    display(df_summary.round(2))
    
    # 最良の実験をハイライト
    if len(df_summary) > 1:
        print("\n🏆 各メトリクスで最良:")
        for col in df_summary.columns:
            best = df_summary[col].idxmax()
            print(f"  {col}: {best} ({df_summary[col].max():.2f}%)")
else:
    print("❌ 比較可能な結果がありません")

## Step 4: オブジェクト別比較

In [ ]:
# MVTecオブジェクトの順序
mvtec_objects = [
    "bottle", "cable", "capsule", "carpet", "grid", "hazelnut",
    "leather", "metal_nut", "pill", "screw", "tile", "toothbrush",
    "transistor", "wood", "zipper"
]

# オブジェクト別AUROCを抽出
object_data = []

for obj in mvtec_objects:
    row = {'Object': obj}
    
    for name, metrics in results.items():
        if obj in metrics and 'classification_AUROC' in metrics[obj]:
            row[name] = metrics[obj]['classification_AUROC'] * 100
        else:
            row[name] = None
    
    object_data.append(row)

if object_data and len(results) > 0:
    df_objects = pd.DataFrame(object_data)
    df_objects = df_objects.set_index('Object')
    
    print("\n" + "="*70)
    print("📋 オブジェクト別 AUROC (%)")
    print("="*70)
    display(df_objects.round(2))
    
    # 差分分析
    if len(results) > 1:
        exp_names = list(results.keys())
        baseline = exp_names[0]
        
        print(f"\n📈 {baseline}からの差分:")
        for exp in exp_names[1:]:
            if exp in df_objects.columns and baseline in df_objects.columns:
                diff = df_objects[exp] - df_objects[baseline]
                improved = diff[diff > 0].sort_values(ascending=False)
                degraded = diff[diff < 0].sort_values()
                
                print(f"\n  {exp}:")
                if len(improved) > 0:
                    print(f"    ✅ 改善: {', '.join([f'{k} (+{v:.1f}%)' for k, v in improved.head(5).items()])}")
                if len(degraded) > 0:
                    print(f"    ⚠️ 低下: {', '.join([f'{k} ({v:.1f}%)' for k, v in degraded.head(5).items()])}")

## Step 5: 可視化

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

if len(results) > 0 and len(object_data) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # グラフ1: オブジェクト別AUROC棒グラフ
    ax1 = axes[0]
    x = np.arange(len(mvtec_objects))
    width = 0.25
    
    exp_names = list(results.keys())
    colors = ['#3498db', '#e74c3c', '#2ecc71']
    
    for i, (name, color) in enumerate(zip(exp_names, colors)):
        if name in df_objects.columns:
            values = df_objects[name].fillna(0).values
            ax1.bar(x + i * width, values, width, label=name, color=color, alpha=0.8)
    
    ax1.set_xlabel('Object')
    ax1.set_ylabel('AUROC (%)')
    ax1.set_title('Object-wise AUROC Comparison')
    ax1.set_xticks(x + width)
    ax1.set_xticklabels(mvtec_objects, rotation=45, ha='right')
    ax1.legend()
    ax1.set_ylim([50, 102])
    ax1.axhline(y=90, color='gray', linestyle='--', alpha=0.5)
    ax1.grid(axis='y', alpha=0.3)
    
    # グラフ2: 平均メトリクス比較
    ax2 = axes[1]
    metrics_names = ['AUROC', 'AP', 'F1']
    x2 = np.arange(len(metrics_names))
    
    for i, (name, color) in enumerate(zip(exp_names, colors)):
        if name in results:
            values = [
                results[name].get('mean_classification_au_roc', 0) * 100,
                results[name].get('mean_classification_ap', 0) * 100,
                results[name].get('mean_classification_f1', 0) * 100,
            ]
            ax2.bar(x2 + i * width, values, width, label=name, color=color, alpha=0.8)
    
    ax2.set_xlabel('Metric')
    ax2.set_ylabel('Score (%)')
    ax2.set_title('Mean Classification Metrics')
    ax2.set_xticks(x2 + width)
    ax2.set_xticklabels(metrics_names)
    ax2.legend()
    ax2.set_ylim([80, 100])
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('experiment_comparison.png', dpi=150)
    plt.show()
    
    print("\n💾 グラフを experiment_comparison.png に保存しました")
else:
    print("❌ 可視化するデータがありません")

## Step 6: 問題カテゴリの詳細分析

In [ ]:
# 問題があるカテゴリ（AUROC < 90%）を特定
print("\n" + "="*60)
print("🔍 問題カテゴリ分析 (AUROC < 90%)")
print("="*60)

for name, metrics in results.items():
    print(f"\n【{name}】")
    problem_objects = []
    
    for obj in mvtec_objects:
        if obj in metrics and 'classification_AUROC' in metrics[obj]:
            auroc = metrics[obj]['classification_AUROC'] * 100
            if auroc < 90:
                problem_objects.append((obj, auroc))
    
    if problem_objects:
        for obj, auroc in sorted(problem_objects, key=lambda x: x[1]):
            status = "⚠️" if auroc >= 70 else "❌"
            print(f"  {status} {obj}: {auroc:.1f}%")
    else:
        print("  ✅ 全カテゴリ 90%以上")

## Step 7: 結論

In [ ]:
print("\n" + "="*60)
print("📝 結論")
print("="*60)

if len(results) > 1:
    exp_names = list(results.keys())
    baseline = exp_names[0]
    baseline_auroc = results[baseline].get('mean_classification_au_roc', 0) * 100
    
    print(f"\nベースライン ({baseline}): AUROC {baseline_auroc:.2f}%")
    print("\n各手法の効果:")
    
    for exp in exp_names[1:]:
        if exp in results:
            exp_auroc = results[exp].get('mean_classification_au_roc', 0) * 100
            diff = exp_auroc - baseline_auroc
            
            if diff > 0.1:
                print(f"  ✅ {exp}: +{diff:.2f}% (AUROC: {exp_auroc:.2f}%)")
            elif diff < -0.1:
                print(f"  ⚠️ {exp}: {diff:.2f}% (AUROC: {exp_auroc:.2f}%)")
            else:
                print(f"  ➡️ {exp}: ±{abs(diff):.2f}% (AUROC: {exp_auroc:.2f}%) - 変化なし")
    
    # 最良モデル
    best_exp = max(results.items(), key=lambda x: x[1].get('mean_classification_au_roc', 0))
    print(f"\n🏆 最良モデル: {best_exp[0]} (AUROC: {best_exp[1].get('mean_classification_au_roc', 0)*100:.2f}%)")
else:
    print("\n比較対象が1つのみです。複数の実験結果を用意してください。")